# Trying out Autogluon for Image Classification

In [1]:
from autogluon.multimodal.utils.misc import shopee_dataset

e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from autogluon.multimodal import MultiModalPredictor
from autogluon.multimodal.presets import get_automm_presets
from IPython.display import Image, display
from ray import tune
from datetime import datetime
from contextlib import redirect_stdout
import json
import yaml
import uuid
import logging
import sys
from io import StringIO


2024-12-04 14:24:57,174	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-04 14:24:57,273	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
download_dir = './ag_automm_tutorial_imgcls'
train_data_path, test_data_path = shopee_dataset(download_dir)

In [6]:
train_data_path

,image,label
0,e:\Current_Workdir\palm-fruit-classification\o...,0
1,e:\Current_Workdir\palm-fruit-classification\o...,0
2,e:\Current_Workdir\palm-fruit-classification\o...,0
3,e:\Current_Workdir\palm-fruit-classification\o...,0
4,e:\Current_Workdir\palm-fruit-classification\o...,0
...,...,...
795,e:\Current_Workdir\palm-fruit-classification\o...,3
796,e:\Current_Workdir\palm-fruit-classification\o...,3
797,e:\Current_Workdir\palm-fruit-classification\o...,3
798,e:\Current_Workdir\palm-fruit-classification\o...,3


In [7]:
test_data_path

,image,label
0,e:\Current_Workdir\palm-fruit-classification\o...,0
1,e:\Current_Workdir\palm-fruit-classification\o...,0
2,e:\Current_Workdir\palm-fruit-classification\o...,0
3,e:\Current_Workdir\palm-fruit-classification\o...,0
4,e:\Current_Workdir\palm-fruit-classification\o...,0
...,...,...
75,e:\Current_Workdir\palm-fruit-classification\o...,3
76,e:\Current_Workdir\palm-fruit-classification\o...,3
77,e:\Current_Workdir\palm-fruit-classification\o...,3
78,e:\Current_Workdir\palm-fruit-classification\o...,3


In [8]:
train_data_path.dtypes

image    object
label     int64
dtype: object

In [9]:
test_data_path

,image,label
0,e:\Current_Workdir\palm-fruit-classification\o...,0
1,e:\Current_Workdir\palm-fruit-classification\o...,0
2,e:\Current_Workdir\palm-fruit-classification\o...,0
3,e:\Current_Workdir\palm-fruit-classification\o...,0
4,e:\Current_Workdir\palm-fruit-classification\o...,0
...,...,...
75,e:\Current_Workdir\palm-fruit-classification\o...,3
76,e:\Current_Workdir\palm-fruit-classification\o...,3
77,e:\Current_Workdir\palm-fruit-classification\o...,3
78,e:\Current_Workdir\palm-fruit-classification\o...,3


In [10]:
hyperparameters, hyperparameter_tune_kwargs = get_automm_presets(problem_type="default", presets="medium_quality_hpo")
print(f"hyperparameters: {yaml.dump(hyperparameters, allow_unicode=True, default_flow_style=False)}")
print(f"hyperparameter_tune_kwargs: {json.dumps(hyperparameter_tune_kwargs, sort_keys=True, indent=4)}")

hyperparameters: env.batch_size: !!python/object:ray.tune.search.sample.Categorical
  categories:
  - 16
  - 32
  - 64
  - 128
  - 256
  sampler: !!python/object:ray.tune.search.sample._Uniform {}
model.document_transformer.checkpoint_name: microsoft/layoutlmv2-base-uncased
model.hf_text.checkpoint_name: !!python/object:ray.tune.search.sample.Categorical
  categories:
  - google/electra-small-discriminator
  - google/flan-t5-small
  - microsoft/deberta-v3-xsmall
  - albert-base-v2
  - microsoft/MiniLM-L12-H384-uncased
  sampler: !!python/object:ray.tune.search.sample._Uniform {}
model.names:
- ft_transformer
- timm_image
- hf_text
- document_transformer
- fusion_mlp
model.timm_image.checkpoint_name: !!python/object:ray.tune.search.sample.Categorical
  categories:
  - mobilenetv3_large_100
  - gluon_resnet18_v1b
  - maxvit_rmlp_pico_rw_256.sw_in1k
  sampler: !!python/object:ray.tune.search.sample._Uniform {}
optimization.learning_rate: !!python/object:ray.tune.search.sample.Float
  lowe

In [11]:
model_path = f"./tmp/{uuid.uuid4().hex}-automm_shopee"

In [12]:
predictor_hpo = MultiModalPredictor(label="label", eval_metric="f1_macro", path=model_path)
predictor_hpo.set_verbosity(4)

hyperparameters = {
            "optimization.learning_rate": tune.uniform(0.00005, 0.001),
            "optimization.max_epochs": tune.randint(10,20),
            "model.timm_image.checkpoint_name": tune.choice(["ghostnet_100",
                                                             "mobilenetv3_large_100"])
}
hyperparameter_tune_kwargs = {
    "searcher": "bayes", # random
    "scheduler": "ASHA",
    "num_trials": 5,
    "num_to_keep": 2,
}

# Run the HPO fit
predictor_hpo.fit(
    train_data=train_data_path,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

save path: e:\Current_Workdir\palm-fruit-classification\old\lemme_learn_the_framework_first\tmp\1b5fc25fe96b45d7a8c4ce0c0a8ce493-automm_shopee
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       111.01 GB / 127.91 GB (86.8%)
Disk Space Avail:   1457.80 GB / 1863.00 GB (78.3%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	4 unique label values:  [0, 1, 2, 3]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])


column_types: OrderedDict([('image', 'image_path'), ('label', 'categorical')])
image columns: ['image']
validation_metric_name: f1_macro
minmax_mode: max
Resources info for NonParallelGpuResourceCalculator: {'resources_per_job': {'cpu': 32, 'gpu': 1}, 'num_parallel_jobs': 1.0, 'batches': 5, 'cpu_per_job': 32, 'gpu_per_job': 1}
resources_per_trial to be dispatched by ray tune: {'cpu': 32, 'gpu': 1}


Trial name,should_checkpoint,val_f1_macro
20dd65f6,True,0.265657
6ddc1dca,True,0.93084
97acd83e,True,0.918638
c9f4aa94,True,0.22842
fc18d177,True,0.222794


Removing non-optimal trials and only keep the best one.
image augmentation space [<identity>, <AutoContrast>, <Equalize>, <Rotate>, <Solarize>, <Color>, <Posterize>, <Contrast>, <Brightness>, <Sharpness>, <ShearX>, <ShearY>, <TranslateX>, <TranslateY>]
output_shape: 4
initializing mobilenetv3_large_100
mix_choice: all_logits
outer layers are treated as head: ['head.weight', 'head.bias']
Start to fuse 3 checkpoints via the greedy soup algorithm.
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\learners\base.py:2111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limi

Predicting DataLoader 0: 100%|██████████| 5/5 [00:00<00:00,  9.07it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = to

Predicting DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 14.18it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = to

Predicting DataLoader 0: 100%|██████████| 5/5 [00:00<00:00, 14.83it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = to

In [14]:
from ray.tune import ExperimentAnalysis

analysis = ExperimentAnalysis("lemme_learn_the_framework_first/tmp/2d1b4acbd01d4663ba8610070b2ca70e-automm_shopee")
analysis

ArrowInvalid: URI has empty scheme: 'lemme_learn_the_framework_first/tmp/2d1b4acbd01d4663ba8610070b2ca70e-automm_shopee'

In [11]:
from ray.tune.analysis import ExperimentAnalysis

# Path to AutoGluon Ray tuning output
analysis = ExperimentAnalysis("E:\\Current_Workdir\\palm-fruit-classification\lemme_learn_the_framework_first\\tmp\\903a01351f4d489581053bd484381b7a-automm_shopee")

# Get all trials and results
for trial in analysis.trials:
    print(f"Trial ID: {trial.trial_id}")
    print(f"Status: {trial.status}")
    print(f"Last Result: {trial.last_result}")
    print(f"Log Directory: {trial.logdir}")

ValueError: No experiment snapshot file of form 'experiment_state-*.json' was found at: (local, E:/Current_Workdir/palm-fruit-classification/lemme_learn_the_framework_first/tmp/903a01351f4d489581053bd484381b7a-automm_shopee)
Please check if you specified the correct experiment path, which should be a combination of the `storage_path` and `name` specified in your run.

In [ ]:
predictor_hpo.fit_summary()

In [ ]:
import json

# Path to the experiment state file
experiment_state_path = "E:\\Current_Workdir\\palm-fruit-classification\\lemme_learn_the_framework_first\\tmp\\903a01351f4d489581053bd484381b7a-automm_shopee\\experiment_state-2024-12-03_15-07-17.json"

# Load the tuning results
with open(experiment_state_path, 'r') as file:
    experiment_data = json.load(file)

# Inspect the experiment data
print(json.dumps(experiment_data, indent=2))


In [ ]:
import json
import pandas as pd

# Path to the experiment state file
experiment_state_path = "E:\\Current_Workdir\\palm-fruit-classification\\lemme_learn_the_framework_first\\tmp\\903a01351f4d489581053bd484381b7a-automm_shopee\\experiment_state-2024-12-03_15-07-17.json"

# Load the tuning results
with open(experiment_state_path, 'r') as file:
    experiment_data = json.load(file)

# Extract trial data from the JSON
trials = experiment_data.get("trials", {})

# Prepare a list to hold parsed trial data
parsed_data = []

# Iterate over trials to extract relevant information
for trial_id, trial_info in trials.items():
    trial_data = {
        "Trial ID": trial_id,
        "Status": trial_info.get("status"),
        "Start Time": trial_info.get("start_time"),
        "End Time": trial_info.get("end_time"),
        "Metric (e.g., val_f1_macro)": trial_info.get("metric_results", {}).get("val_f1_macro"),
    }
    # Add additional details if needed (e.g., hyperparameters)
    hyperparams = trial_info.get("config", {})
    for key, value in hyperparams.items():
        trial_data[f"Hyperparam: {key}"] = value
    
    # Append parsed trial data
    parsed_data.append(trial_data)

# Create a Pandas DataFrame from the parsed data
df = pd.DataFrame(parsed_data)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file if needed
df.to_csv("parsed_experiment_data.csv", index=False)


In [ ]:
import pickle

# Define the file path
file_path = r"E:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\903a01351f4d489581053bd484381b7a-automm_shopee\searcher-state-2024-12-03_15-07-17.pkl"

# Open the file and load its content
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now you can use 'data', which contains the contents of the pickle file
print(data)


In [ ]:
import pickle

# Define the file path
file_path = r"E:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\903a01351f4d489581053bd484381b7a-automm_shopee\tuner.pkl"

# Open the file and load its content
with open(file_path, 'rb') as file:
    tuner_data = pickle.load(file)

# Now you can use 'tuner_data', which contains the contents of the pickle file
print(tuner_data)


In [ ]:
import pickle

# Define the file path
file_path = r"E:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\903a01351f4d489581053bd484381b7a-automm_shopee\params.pkl"

# Open the file and load its content
with open(file_path, 'rb') as file:
    params_data = pickle.load(file)

# Now you can use 'params_data', which contains the contents of the pickle file
print(params_data)


In [ ]:
import json

# Define the file path
file_path = r"E:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\903a01351f4d489581053bd484381b7a-automm_shopee\experiment_state-2024-12-03_15-07-17.json"

# Open the file and load its content
with open(file_path, 'r') as file:
    experiment_data = json.load(file)

# Now you can use 'experiment_data', which contains the contents of the JSON file
print(experiment_data)

In [ ]:
print(logs.leaderboard())

In [ ]:
fit_summary = predictor_hpo.fit_summary(verbosity=4, show_plot=True)

In [ ]:
print(fit_summary)

In [ ]:
print(fit_summary.values)

In [ ]:
predictor_hpo

In [ ]:
predictor = MultiModalPredictor.load("E:/Current_Workdir/palm-fruit-classification/lemme_learn_the_framework_first/tmp/744330067ebe4b97b0b22fcf1b4d0fcf-automm_shopee")

Load pretrained checkpoint: E:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\744330067ebe4b97b0b22fcf1b4d0fcf-automm_shopee\model.ckpt


In [ ]:
help(predictor)

Help on MultiModalPredictor in module autogluon.multimodal.predictor object:

class MultiModalPredictor(builtins.object)
 |  MultiModalPredictor(label: 'Optional[str]' = None, problem_type: 'Optional[str]' = None, query: 'Optional[Union[str, List[str]]]' = None, response: 'Optional[Union[str, List[str]]]' = None, match_label: 'Optional[Union[int, str]]' = None, presets: 'Optional[str]' = None, eval_metric: 'Optional[Union[str, Scorer]]' = None, hyperparameters: 'Optional[dict]' = None, path: 'Optional[str]' = None, verbosity: 'Optional[int]' = 2, num_classes: 'Optional[int]' = None, classes: 'Optional[list]' = None, warn_if_exist: 'Optional[bool]' = True, enable_progress_bar: 'Optional[bool]' = None, pretrained: 'Optional[bool]' = True, validation_metric: 'Optional[str]' = None, sample_data_path: 'Optional[str]' = None)
 |  
 |  AutoMM is designed to simplify the fine-tuning of foundation models
 |  for downstream applications with just three lines of code.
 |  AutoMM seamlessly integr

In [ ]:
scores = predictor.evaluate(test_data_path, metrics=["f1_macro"])
print(scores)

In [ ]:
image_path = test_data_path.iloc[0]['image']
pil_img = Image(filename=image_path)
display(pil_img)

In [ ]:
predictions = predictor.predict({'image': [image_path]})
print(predictions)

In [ ]:
proba = predictor.predict_proba({'image': [image_path]})
print(proba)

In [ ]:
feature = predictor.extract_embedding({'image': [image_path]})
print(feature[0].shape)

In [ ]:
# predictor = MultiModalPredictor.load("e:\Current_Workdir\palm-fruit-classification\lemme_learn_the_framework_first\tmp\43522911e2a447a6b911e0a9e4ff3c0b-automm_shopee")

In [ ]:
predictor.dump_model("./tmp/final")